# Instructions

Start a Jupyter Notebook using any platform that you are comfortable with and do the following:

* Write some markdown to explain that this notebook will be mainly used for the capstone project.  
* Import the pandas library as pd.  
* Import the Numpy library as np.  
* Print the following the statement: Hello Capstone Project Course!  
* Push the Notebook to your Github repository and submit a link to the notebook on your Github repository.  

## This notebook will be used for the Capstone Project from the 
## "Applied Data Science Capstone" IBM course

In [1]:
import pandas as pd
import numpy as np

In [2]:
print('Hello Capstone Project Course!')

Hello Capstone Project Course!
